# ATEK Data Proprocessing Example

### 1. Load the example adt data

In [1]:
import os

from atek.data_preprocess.sample_builders.cubercnn_sample_builder import CubeRCNNSampleBuilder
from atek.data_preprocess.wds_writer import AtekWdsWriter

from omegaconf import OmegaConf

example_adt_data_dir = "/home/louy/Calibration_data_link/Atek/2024_01_25_AseTest/adt_data_example/1WM103600M1292_optitrack_SSL_1_Aria_OS_DinoToy_seq010"
example_ase_data_dir = "/home/louy/Calibration_data_link/Atek/2024_05_07_EfmDataTest/ase_data_example/euston_simulation_100077_device0"
config_path = os.path.join("/home/louy/Calibration_data_link/Atek/2024_05_28_CubeRcnnTest/cubercnn_preprocess_ase_config.yaml")

conf = OmegaConf.load(config_path)

### 2. Sample builder example

In [ ]:
sample_builder = CubeRCNNSampleBuilder(
            conf=conf.processors,
            vrs_file=os.path.join(example_ase_data_dir, "video.vrs"),
            mps_files={
                "mps_closedloop_traj_file": os.path.join(
                    example_ase_data_dir, "test_ADT_trajectory.csv"
                ),
            },
            gt_files={
                "obb3_file": os.path.join(example_ase_data_dir, "3d_bounding_box.csv"),
                "obb3_traj_file": os.path.join(
                    example_ase_data_dir, "scene_objects.csv"
                ),
                "obb2_file": os.path.join(example_ase_data_dir, "2d_bounding_box.csv"),
                "instance_json_file": os.path.join(
                    example_ase_data_dir, "instances.json"
                ),
            },
        )

# Print the frame dataclass keys
sample = sample_builder.get_sample_by_timestamp_ns(10_000_000_000)
from dataclasses import asdict
print(asdict(sample).keys())

### 2. Write wds files

In [ ]:
from atek.data_preprocess.wds_writer import (
    AtekWdsWriter,
)
from atek.data_preprocess.subsampling_lib.temporal_subsampler import CameraTemporalSubsampler

subsampler = CameraTemporalSubsampler(vrs_file = os.path.join(example_ase_data_dir, "video.vrs"), conf = conf.camera_temporal_subsampler)

conf.wds_writer.prefix_string = "test"
atek_wds_writer = AtekWdsWriter(output_path = "/home/louy/Calibration_data_link/Atek/2024_05_28_CubeRcnnTest/wds_output/test_cubercnn_ase_1", conf = conf.wds_writer)

for i in range(subsampler.get_total_num_samples()):
    timestamps_ns = subsampler.get_timestamps_by_sample_index(i)
    print(f"sample {i} has timestamps of {timestamps_ns}")
    
    for t in timestamps_ns:
        sample = sample_builder.get_sample_by_timestamp_ns(t)
        if sample is not None:
            atek_wds_writer.add_sample(data_sample = sample)

atek_wds_writer.close()
